## DSPy - Declaring Self Improving Language Programs Pythonically

![](banner.png)

### Introduction and the need for DSPy

DSPy is a framework for developing and optimizing LLM programs rather than manually tweaking prompts.

A solution to a single application/usecase with an LLM is not solved by training/prompting the model to solve the entire problem at once (giving the whole input) and expecting an output but rather is solved by breaking the program into multiple sub problems and then asking the LLMs to solve each like divide and conquer.

**While doing so, there will be some fuzzy steps (not hard coded steps) that are not standard or not well defined, but affects the output of the model like prompts. Now, getting out of toy example, the models trying to solve real world examples are very sensitive to prompts.**

So, we have DSPy that deals with optimizations these fuzzy steps given a model & given a usecase/problem statement. The core idea is to program - not prompting language models.

![](workflow.png)


In DSPy, we dont write and tweak brittle prompts rather we write a short langauge program in python that designs a prompt to use LM effectively. This means that we are compiling a python code (here: high level langauge) to a natural language (here: low level language).

So, there are two running ideas in DSPy:
1. We seperate the program logic (executable modules) from how a language model is taught to produce an output
2. We optimize how the LMs are prompted (or finetuned) via discrete search over modular language elements. 

> _DSPy is a framework for algorithmically optimizing LM prompts and weights, especially when LMs are used one or more times within a pipeline. To use LMs to build a complex system without DSPy, you generally have to: (1) break the problem down into steps, (2) prompt your LM well until each step works well in isolation, (3) tweak the steps to work well together, (4) generate synthetic examples to tune each step, and (5) use these examples to finetune smaller LMs to cut costs. Currently, this is hard and messy: every time you change your pipeline, your LM, or your data, all prompts (or finetuning steps) may need to change._

Existing LM pipelines are modular in principle (like with modular RAGs), but extremely messy and ad-hoc in practice because these fuzzy steps are one of the key factors in development and it differs for every model >> because instruction tuning of the model might have occured in different ways, the system context might have been configured in different ways (config layer of LLMs) and more. 

**It is not always sure that having a better prompt for a model does not mean it will work better for other different models available.** >> Why is this the case? :: Because models don't learn the english that we do, a subtle change in the language wording makes a big difference to their understanding and effeectiveness.



For example:

Maybe a simple answer the question based on a given context works for GPT4 becuase its intelligent enough (and massively trained) but if you are working with open-source llms (while are still dragging behind) you might want to give it some examples (few shots) for it to perform better. ***This means that we are solving the same problem but needs different prompts that gives three different results.**

## Working and Understanding DSPy

DSPy introduces signatures (to abstract prompts), modules (to abstract prompting techniques), and optimizers that can tune the prompts (or weights) of modules.

1. Signatures


>_A signature is a declarative specification of input/output behavior of a DSPy module. Signatures allow you to tell the LM what it needs to do, rather than specify how we should ask the LM to do it._

Examples of signatures:
-  question -> reasoning, answer
-  document -> summary
-  context, question -> search_query
-  question, context -> reasoning, answer

The DSPy compiler will figure out how to build a highly-optimized prompt for your LM (or finetune your small LM) for your signature, on your data, and within your pipeline. 

![](DSPy.png)

_Figure 2: Understanding of DSPy with comparison of Pytorch_

2. Optimizations

>_An optimizer is an algorithm that can tune DSPy program parameters (i.e., the prompts in our cases here, but can be LMs weights as well) to maximize a certain evaluation metrics, like accuracy._

Optimization problem is : Take this program (flow of the program), here are 20 questions and its answers, metric is : does the answer match ? :: find me the best prompt that helps me get that.


So, Keep a record of which and what prompt worked in the past, and use that prompt for generation (as few shots) >>> Generate >>> filter (discrete search) >>> use >>> The only problem becomes how to not do all the combination >>> Optimize

Inner workings of optimizations:

1. For given questions and answer pairs, pass it through the model >>> Run the pair through your pipeline
2. Keep a record of the traces (Q/A pairs) that perform good in the evaluation metric >> acheive high quality
3. Use those pairs as a few shot examples to see if it works well or not? Find the traces that works well : Optimization
4. The core optimization is : how not to go through all the traces combination, rather select the most effective ones? >> Selecting from a discrete set of examples pairs >> Discrete Optimization

### Analogy to a Neural network training


> When we build neural networks, we don't write manual for-loops over lists of hand-tuned floats. Instead, you might use a framework like PyTorch to compose layers (e.g., Convolution or Dropout) and then use optimizers (e.g., SGD or Adam) to learn the parameters of the network.



TLDR;

The idea is not to write chain of thought in prompt but to rather use a module that already has the COT like langchainhub and optimize the prompt based on your dataset.

### An approach to RAG system with DSPy


In [9]:
import dspy
import os

# ENVIRONMENTS
from dotenv import load_dotenv
load_dotenv("./.env")


# Wrap the AzureOpenAI LLM using DSPy
turbo = dspy.AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_base=os.environ["AZURE_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
    deployment_id = os.environ["DEPLOYMENT_NAME"],
    max_tokens=250
    )

# Original Dataset / Dataset in which we are expected to retreive and provide context from
# RAG document source
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)
# lm = Language Model
# rm = Retreival Model (but we are going to use our own retreival approach)

**An example of a single entry in colbertv2_wiki17_abstracts**
```
{
    'text': 'David (commentator) | David (Greek: Δαυΐδ ; fl. 6th century) was a Greek scholar and a commentator on Aristotle and Porphyry.', 
    'pid': 4948591, 
    'rank': 1, 
    'score': 27.326705932617188, 
    'prob': 0.296344890368164, 
    'long_text': 'David (commentator) | David (Greek: Δαυΐδ ; fl. 6th century) was a Greek scholar and a commentator on Aristotle and Porphyry.'
    }
```

We also have a HotPotQA dataset that contains QA pairs to perform complex reasoning and provide explanations for answers. This dataset is designed to evaluate LMs on its generation task.

In [10]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

**An example of a train data in HotPotQA is:**

In [11]:
dataset.train[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys=None)

**An example of a train data in HotPotQA is:**

In [12]:
dataset.dev[0]

Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 'gold_titles': {'Qionghai', 'Cangzhou'}}) (input_keys=None)

We will now create a retreival method that is supposed to replicate the retreival mechanism in a RAG system.

This `retreival` method given a query, provides that topk results from an API endpoint.

In [13]:
import requests

def retreive(query):
    print("---RETREIVAL---")
    # Given a query string, get the top 5 most similar abstracts to query
    payload = {"query": query, "k": 5}

    # source endpoint
    url='http://20.102.90.50:2017/wiki17_abstracts'
    
    # request
    res = requests.get(url, params=payload, timeout=10)

    # topk filtering
    topk = res.json()["topk"][:5]
    topk = [{**d, "long_text": d["text"]} for d in topk]

    # context creation    
    contexts = [" ".join([k["text"], k["long_text"]]) for k in topk]
    
    return contexts

Now that we have our retreival method ready, we will start creating the DSPy program.

We will start by designing a signature : `context, question -> answer` , that takes in a context and a question to provide an answer.

In [14]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [15]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        # Retreival model using DSPy
        # self.retrieve = dspy.Retrieve(k=num_passages)

        # Apply Chain of Thought while generating the answer
        # This basically means, the prompt will be designed in a CoT manner that takes in the context, question -> answer
        # To understand this refer to the generated prompt in the coming blocks
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        # context = self.retrieve(question).passages
        # we will not follow the DSPy way of retreival but rather have our own retreival approach
        # The general idea remains tha same        
        context = retreive(question)

        # generate the prediction
        print("---GENERATION---")
        prediction = self.generate_answer(context=context, question=question)

        # return an object of Prediction
        return dspy.Prediction(context=context, answer=prediction.answer)

We now have a RAG system in DSPy that takes in a question, retreives contexts and generates the answer. Let us make a invocation (if you are coming from langchain) or call the model (if you are coming from training a deep neural network in Pytorch).

In [16]:
# Lets take a toy question and pass it through our RAG system to get the answer
TOY_QUERY = "What castle did David Gregory inherit?"

RAG().forward(question=TOY_QUERY).answer

---RETREIVAL---
---GENERATION---


'Kinnairdy Castle'

In [17]:
# To see the context for the generation we have:
RAG().forward(question=TOY_QUERY).context

---RETREIVAL---
---GENERATION---


['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted. David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton de

Did you just see what happened? For those who are familiar to a RAG application or ones who create LLM based application, we never wrote any prompt. Did we? yes we didn't. Then how did the LLM generate the response and a perfect response for that matter.

This is where DSPy is doing wonders, it created a prompt on its own based on the following program:
```
self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
```

But it is impirical for an LLM to have a prompt, where is out prompt then? lets take a look into it:

In [18]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory's use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted. David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scot

Just looking at the prompt, it follows the CoT indeed and my god its a well crafted prompt. 

I know I know I know, its long >> too many tokens >> expensive per call cost >> well its a tradeoff to quality of generation.

Definitely will be more relevant if we go with open-source LLMs.

### DSPy Optimization 

**But WAIT, where is optmization?**

Now here, the optimization is basically making this prompt more better, adding few shot examples to how we want the output to be like and making our RAG system more reobust. 

_How do we know which prompt is better?_
> Evaluation metrics like `exact_match`, `partial_match`

Given out QA pairs, we will create a teleprompter or an optimizer, that optimally selects few examples (the ones that give us best LLM generated output) and add it to the prompt.

_What is optimially select?_

>

    If you have very little data, e.g. 10 examples of your task, use BootstrapFewShot.

    If you have slightly more data, e.g. 50 examples of your task, use BootstrapFewShotWithRandomSearch.

    If you have more data than that, e.g. 300 examples or more, use MIPRO.

    If you have been able to use one of these with a large LM (e.g., 7B parameters or above) and need a very efficient program, compile that down to a small LM with BootstrapFinetune.


In [19]:
# Here we only have a few examples so we will use BootstrapFewShot
# The evaluation metric is a function that takes in an example (gold output) and a prediction from the model to give a numeric score

from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

  0%|                                                                                       | 0/20 [00:00<?, ?it/s]

---RETREIVAL---


  5%|███▉                                                                           | 1/20 [00:00<00:18,  1.02it/s]

---GENERATION---
---RETREIVAL---


 10%|███████▉                                                                       | 2/20 [00:01<00:17,  1.04it/s]

...

---GENERATION---
---RETREIVAL---


 45%|███████████████████████████████████▌                                           | 9/20 [00:08<00:10,  1.02it/s]

---GENERATION---
---RETREIVAL---


 50%|███████████████████████████████████████                                       | 10/20 [00:09<00:09,  1.04it/s]

---GENERATION---
Bootstrapped 4 full traces after 11 examples in round 0.


After 11 examples, running in sequence, 4 samples are added in the prompt (bootstrapped).

In [20]:
# let's answer the same question again
TOY_QUERY = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(TOY_QUERY)

# Print the contexts and the answer.
print(f"Question: {TOY_QUERY}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

---RETREIVAL---
---GENERATION---
Question: What castle did David Gregory inherit?
Predicted Answer: Kinnairdy Castle
Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...', 'David Gregory (historian) | David Gregory (1696–1767) was an English churchman and academic, Dean of Christ Church, Oxford and the first Regius Professor of Modern History at Oxford. David Gregory (hi...', 'Gregory of Gaeta | Gr

Now, lets look how out prompt is like:

In [21]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: In what year was the club founded that played Manchester City in the 1972 FA Charity Shield
Answer: 1874

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas fo

Now, thats an optimized RAG with a well structured prompt. 

**We didn't have to write any CoT prompts, nor we had to explicitly mention the type of output we want. All of those fuzzy steps were successfully programmed.**

### Evaluation of RAG systems

Okay, then how do we analyze this RAG system we designed?

We can use the `dev` set we created earlier with DSPy `evaluate`.

Lets run an evaluation pipeline on : `compiled_rag` (original)


In [22]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

  0%|                                                                                       | 0/50 [00:00<?, ?it/s]

---RETREIVAL---


Average Metric: 0 / 1  (0.0):   2%|▉                                                | 1/50 [00:01<00:51,  1.06s/it]

---GENERATION---
---RETREIVAL---


Average Metric: 1 / 2  (50.0):   4%|█▉                                              | 2/50 [00:02<00:51,  1.06s/it]

...

Average Metric: 26 / 50  (52.0): 100%|█████████████████████████████████████████████| 50/50 [00:49<00:00,  1.00it/s]

---GENERATION---
Average Metric: 26 / 50  (52.0%)



/home/kamal/Downloads/dspy-wiki/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","Yes, Cangzhou is in the Hebei province, but Qionghai is not.",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,✔️ [True]


52.0

Here on the `dev` set, the RAG system was able to acheive the accuracy of `52%`, which shows that there are a lot of improvements required in the retreival process.

### References:

1. [DSPy Documentation](https://dspy-docs.vercel.app/docs/intro)
2. [HOTPOTQA: A Dataset for Diverse, Explainable Multi-hop Question Answering](https://nlp.stanford.edu/pubs/yang2018hotpotqa.pdf)
3. [Intro to DSPy: Goodbye Prompting, Hello Programming!](https://towardsdatascience.com/intro-to-dspy-goodbye-prompting-hello-programming-4ca1c6ce3eb9)
4. [ Optimization of LLM Systems with DSPy and LangChain/LangSmith ](https://www.youtube.com/watch?v=4EXOmWeqXRc)
5. [ A brief overview of Stanford's DSPy - explained simply ](https://www.youtube.com/watch?v=njVKMqs9lxU)
6. [ AI Prompt Engineering Is Dead Long live AI prompt engineering](https://spectrum.ieee.org/prompt-engineering-is-dead)
7. [ SBTB23: Omar Khattab, DSPy: Compiling Declarative Language Model Calls into Self-Improving Pipelines ](https://www.youtube.com/watch?v=Dt3H2ninoeY)
8. [ DSPy: Compiling Declarative Language Model Calls into State-of-the-Art Pipelines ](https://www.youtube.com/watch?v=im7bCLW2aM4)

 Thank you for reading !

The blog is originally published in [Understanding DSPy with RAG - Blogs, Kamal Shrestha](https://shresthakamal.com.np/blog/2024/newsletter-edition-2/).

If you have any feedbacks or suggestions , please do comment.

You can find me on Linkedin and my website.